In [4]:
import pandas as pd
import re

def preprocess(x):
    x = str(x) #轉str
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 去除換行符號
    return str.strip(x)

# 載入公司寶典
公司寶典 = pd.read_excel('..\data\寶典\台塑出口公司名稱及統編.xlsx')[['代號','公司英文名稱']]
公司寶典['公司英文名稱'] = 公司寶典['公司英文名稱'].apply(lambda x:preprocess(x))
print(公司寶典.shape)
公司寶典['尾綴'] = 公司寶典['公司英文名稱'].apply(lambda x:x.split(' ')[-1])
公司寶典['公司英文名稱'] = 公司寶典['公司英文名稱'].apply(lambda x:' '.join(x.split(' ')[:-1]))
公司寶典['公司英文名稱'] = 公司寶典['公司英文名稱'].apply(lambda x:x.replace('CO',''))
公司寶典.loc[公司寶典['尾綴']=='LTD','尾綴'] = 'CO LTD'
display(公司寶典)
print(公司寶典.shape)
公司寶典.to_csv('../data/寶典/公司寶典加尾綴.csv')

(28, 2)


,代號,公司英文名稱,尾綴
0,1,FORMOSA PLASTICS,CORPORATION
1,2,NAN YA PLASTICS,CORPORATION
2,4,FORMOSA CHEMICALS AND FIBRE,CORP
3,6,FORMOSA PETROCHEMICAL,CORP
4,AF,FORMOSA AUTOMOBILE SALES,CORPORATION
5,AH,FORMOSA ASAHI SPANDEX,CO LTD
6,AL,FORMOSA DAIKIN ADVANCED CHEMICALS,CO LTD
7,BM,FORMOSA BIOMEDICAL TECHNOLOGY,CORPORATION
8,BN,HONG JING RESOURCES,CO LTD
9,GF,CHANG GUNG MEDICAL TECHNOLOGY,CO LTD


(28, 3)


In [5]:
#ok